<a href="https://colab.research.google.com/github/TimH2024/MSC-M5-Project/blob/main/5_Hyper_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Hyper Parameter Tuning Results

In [ ]:
pip install tensorflow keras keras-tuner numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
# Core libraries
import numpy as np
import pandas as pd

# Machine learning and preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# TensorFlow/Keras for deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Keras Tuner for hyperparameter tuning
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

In [ ]:
import os
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Define file path for 'filtered_dataset120ML.csv' in the subdirectory
file_path = os.path.join(subdirectory, 'filtered_dataset120ML.csv')

# Check if the file exists, then load it
if os.path.exists(file_path):
    filtered_dataset120 = pd.read_csv(file_path)
    print(f"File 'filtered_dataset120ML.csv' loaded successfully!")
    print(f"DataFrame shape: {filtered_dataset120.shape}")
else:
    print(f"File 'filtered_dataset120ML.csv' not found in '{subdirectory}'. Please check the file path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File 'filtered_dataset120ML.csv' loaded successfully!
DataFrame shape: (102229, 58)


In [ ]:
import os
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Define file path for 'master_results.csv' in the subdirectory
master_results_file_path = os.path.join(subdirectory, 'master_results.csv')

# Check if the file exists, then load it
if os.path.exists(master_results_file_path):
    master_results = pd.read_csv(master_results_file_path)
    print(f"File 'master_results.csv' loaded successfully!")
    print(f"DataFrame shape: {master_results.shape}")
else:
    print(f"File 'master_results.csv' not found in '{subdirectory}'. Please check the file path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File 'master_results.csv' loaded successfully!
DataFrame shape: (16, 10)


In [ ]:
print(master_results)

       Target                        Model  Train R²   Test R²           MAE  \
0   new_price            Linear Regression  0.601408  0.595691  4.716642e-01   
1   new_price      Random Forest Regressor  0.979135  0.850895  2.333770e-01   
2   new_price            XGBoost Regressor  0.832455  0.778517  3.312516e-01   
3   new_price  Gradient Boosting Regressor  0.772447  0.752945  3.536000e-01   
4          PI            Linear Regression  1.000000  1.000000  7.054472e-16   
5          PI      Random Forest Regressor  1.000000  1.000000  3.067508e-14   
6          PI            XGBoost Regressor  0.999957  0.999957  6.001523e-03   
7          PI  Gradient Boosting Regressor  1.000000  1.000000  1.008244e-08   
8   new_price                    Base LSTM  0.668216  0.666281  1.425859e+06   
9          PI                    Base LSTM  0.999555  0.999558  9.299883e-02   
10  new_price                Enhanced LSTM  0.686986  0.679755  1.390046e+06   
11         PI                Enhanced LS

In [ ]:
# Define the feature matrix (X) and target variables (y)

features = filtered_dataset120.drop(columns=['new_price', 'PI'])
targets_120 = filtered_dataset120[['new_price', 'PI']]

X = features.copy()
y = targets_120.copy()

In [ ]:
filtered_dataset120.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102229 entries, 0 to 102228
Data columns (total 58 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   saleable_area(ft^2)                             102229 non-null  int64  
 1   floor                                           102229 non-null  float64
 2   CG                                              102229 non-null  float64
 3   CI                                              102229 non-null  float64
 4   CPI                                             102229 non-null  float64
 5   GDP                                             102229 non-null  float64
 6   HS                                              102229 non-null  float64
 7   IR                                              102229 non-null  float64
 8   LTV                                             102229 non-null  float64
 9   M3                        

# TUNING

In [ ]:
import os
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from keras_tuner import RandomSearch
import numpy as np
import tensorflow as tf

# File paths
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'
master_results_file_path = os.path.join(subdirectory, 'master_results.csv')

# Load or initialize master results table
if os.path.exists(master_results_file_path):
    master_results = pd.read_csv(master_results_file_path)
    print(f"[INFO] Loaded existing master results table from '{master_results_file_path}'.")
else:
    master_results = pd.DataFrame(columns=["Target", "Model", "Type", "Train R²", "Test R²", "MAE", "MSE", "RMSE", "MAPE", "Comments"])
    print(f"[INFO] Initialized a new master results table.")

# Define features and target variables
features = filtered_dataset120.drop(columns=['new_price', 'PI'])
targets_120 = filtered_dataset120[['new_price', 'PI']]
X = features.copy()
y = targets_120.copy()

# Normalize features and targets
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

target_scalers = {col: StandardScaler() for col in targets_120.columns}
y_scaled = pd.DataFrame({col: target_scalers[col].fit_transform(targets_120[[col]]).flatten() for col in targets_120.columns})

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Subset the dataset for faster tuning
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
X_train_sample_lstm = X_train_sample.reshape((X_train_sample.shape[0], 1, X_train_sample.shape[1]))

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    num_layers = hp.Int('num_layers', min_value=1, max_value=3, step=1)

    for i in range(num_layers):
        return_sequences = i < (num_layers - 1)
        model.add(LSTM(
            units=hp.Int(f'units_layer_{i+1}', min_value=32, max_value=128, step=32),
            activation='relu',
            return_sequences=return_sequences,
            input_shape=(X_train_sample_lstm.shape[1], X_train_sample_lstm.shape[2]) if i == 0 else None
        ))
        model.add(Dropout(rate=hp.Choice(f'dropout_layer_{i+1}', values=[0.2, 0.3, 0.4])))

    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])
    ), loss='mse')
    return model

# Initialize tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='lstm_hyper_tuning'
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Loop through targets (new_price and PI) to dynamically update results
for target_name in ["new_price", "PI"]:
    if target_name not in y_train.columns:
        raise KeyError(f"Target column '{target_name}' not found in y_train. Available columns are: {y_train.columns.tolist()}")

    # Train and evaluate the model for the current target
    y_train_target_sample = y_train_sample[[target_name]].values

    # Hyperparameter tuning
    tuner.search(
        X_train_sample_lstm,
        y_train_target_sample,
        validation_split=0.1,
        epochs=20,
        batch_size=64,
        callbacks=[early_stopping]
    )

    # Build and train the best model
    best_hps = tuner.get_best_hyperparameters(1)[0]
    best_model = tuner.hypermodel.build(best_hps)
    history = best_model.fit(
        X_train_lstm,
        y_train[[target_name]].values,
        validation_data=(X_test_lstm, y_test[[target_name]].values),
        epochs=20,
        batch_size=64,
        callbacks=[early_stopping]
    )

    # Evaluate the model
    train_predictions = best_model.predict(X_train_lstm)
    test_predictions = best_model.predict(X_test_lstm)

    scaler_y = target_scalers[target_name]
    train_predictions_rescaled = scaler_y.inverse_transform(train_predictions)
    test_predictions_rescaled = scaler_y.inverse_transform(test_predictions)

    y_train_rescaled = scaler_y.inverse_transform(y_train[[target_name]])
    y_test_rescaled = scaler_y.inverse_transform(y_test[[target_name]])

    train_r2 = r2_score(y_train_rescaled, train_predictions_rescaled)
    test_r2 = r2_score(y_test_rescaled, test_predictions_rescaled)
    mae = mean_absolute_error(y_test_rescaled, test_predictions_rescaled)
    mse = mean_squared_error(y_test_rescaled, test_predictions_rescaled)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_test_rescaled - test_predictions_rescaled) / y_test_rescaled)) * 100

    # Add results to the master results table
    new_results = {
        "Target": target_name,
        "Type": "Neural Network",
        "Train R²": train_r2,
        "Test R²": test_r2,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape,
        "Comments": "Working Well" if test_r2 > 0.75 and mape < 10 else "Needs Improvement"
    }

    # Convert new_results to a DataFrame
    new_results_df = pd.DataFrame([new_results])

    # Check if the target and model already exist in the master results table
    existing_row_index = master_results[
        (master_results["Target"] == new_results["Target"]) & (master_results["Model"] == new_results["Model"])
    ].index

    if len(existing_row_index) > 0:
        # Update the existing row by replacing its values
        master_results.loc[existing_row_index, :] = new_results_df.iloc[0].values
    else:
        # Append the new results
        master_results = pd.concat([master_results, new_results_df], ignore_index=True)

# Save updated results
master_results.to_csv(master_results_file_path, index=False)
print(f"[INFO] Updated master results saved to '{master_results_file_path}'.")

# Display the final results table
print("\nFinal Master Results Table:")
print(master_results.to_string(index=False))

[INFO] Loaded existing master results table from '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/master_results.csv'.
Reloading Tuner from hyperparam_tuning/lstm_hyper_tuning/tuner0.json


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 35s 19ms/step - loss: 0.5877 - val_loss: 0.3600
Epoch 2/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.3911 - val_loss: 0.3448
Epoch 3/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.3794 - val_loss: 0.3456
Epoch 4/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.3686 - val_loss: 0.3359
Epoch 5/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.3690 - val_loss: 0.3352
Epoch 6/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.3644 - val_loss: 0.3315
Epoch 7/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - loss: 0.3646 - val_loss: 0.3321
Epoch 8/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - loss: 0.3567 - val_loss: 0.3321
Epoch 9/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.3577 - val_loss: 0.3315
2556/2556 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
639/639 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1/20


<ipython-input-18-dc32087b14d5>:155: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Neural Network' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  master_results.loc[existing_row_index, :] = new_results_df.iloc[0].values
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1278/1278 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - loss: 0.1992 - val_loss: 6.6580e-04
Epoch 2/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0211 - val_loss: 3.4260e-04
Epoch 3/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0190 - val_loss: 2.5618e-04
Epoch 4/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0181 - val_loss: 3.7161e-04
Epoch 5/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0177 - val_loss: 3.9568e-04
Epoch 6/20
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0174 - val_loss: 4.4070e-04
2556/2556 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
639/639 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
[INFO] Updated master results saved to '/content/drive/My Drive/Colab Notebooks/M5 Code and Data/master_results.csv'.

Final Master Results Table:
   Target                       Model       Train R²  Test R²          MAE          MSE         RMSE         MAPE          Comments              Type
new_price           Linear Regression       0.601408 0.595691 4.716642e-

# BEST HYPER TUNED ARCHITECTURE

In [ ]:
# Display the best architecture from the hyperparameter tuner
print("[INFO] Best Hyperparameters for the Tuned LSTM Model:")
print(f"Number of Layers: {best_hps.get('num_layers')}")

for i in range(best_hps.get('num_layers')):
    units = best_hps.get(f'units_layer_{i+1}')
    dropout = best_hps.get(f'dropout_layer_{i+1}')
    print(f"Layer {i+1}: {units} units, Dropout rate: {dropout}")

learning_rate = best_hps.get('learning_rate')
print(f"Learning Rate: {learning_rate}")

# Alternatively, show a summary of the best model architecture
print("\n[INFO] Best Model Architecture:")
best_model.summary()

[INFO] Best Hyperparameters for the Tuned LSTM Model:
Number of Layers: 3
Layer 1: 96 units, Dropout rate: 0.4
Layer 2: 32 units, Dropout rate: 0.2
Layer 3: 32 units, Dropout rate: 0.2
Learning Rate: 0.001

[INFO] Best Model Architecture:


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                       │ (None, 1, 96)               │          58,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 1, 96)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 1, 32)               │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 1, 32)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 250,853 (979.90 KB)

 Trainable params: 83,617 (326.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 167,236 (653.27 KB)

# Save the Dataframe & Data

In [ ]:
import os
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the subdirectory path
subdirectory = '/content/drive/My Drive/Colab Notebooks/M5 Code and Data'

# Define file path for 'filtered_dataset120ML.csv' in the subdirectory
file_path = os.path.join(subdirectory, 'filtered_dataset120ML.csv')

# Check if the file exists, then load it
if os.path.exists(file_path):
    filtered_dataset120 = pd.read_csv(file_path)
    print(f"File 'filtered_dataset120ML.csv' loaded successfully!")
    print(f"DataFrame shape: {filtered_dataset120.shape}")
else:
    print(f"File 'filtered_dataset120ML.csv' not found in '{subdirectory}'. Please check the file path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File 'filtered_dataset120ML.csv' loaded successfully!
DataFrame shape: (102229, 58)


In [ ]:
print(master_results.head(20))
print(master_results.shape)